In [ ]:
from google.colab import files
uploaded = files.upload()

Saving merge_full.csv to merge_full.csv


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections
import re
import networkx
import numpy as np
import ast
import nltk
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
#from gensim.models.phrases import Phrases, ENGLISH_CONNECTOR_WORDS

In [ ]:
total_filename = '/content/merge_full.csv'
filename_export = 'merge_clean_v3.csv'

In [ ]:
df = pd.read_csv(total_filename, encoding="utf-8")
df

,username,created_at,tweet,verdict_neilson,verdict_michelle,verdict_liecel
0,PIACaraga,2021-09-21 23:53:36+00:00,"RT @pcoogov: TINGNAN: Kahapon, Setyembre 20, i...",INVALID,TRUE,TRUE
1,PIACaraga,2021-09-21 23:53:27+00:00,RT @pcoogov: TINGNAN: Nakatanggap ng additiona...,INVALID,TRUE,TRUE
2,PIACaraga,2021-09-21 23:53:17+00:00,"RT @pcoogov: TINGNAN: Nitong Lunes, Setyembre ...",INVALID,INVALID,TRUE
3,PIACaraga,2021-09-21 23:52:36+00:00,RT @pcoogov: TINGNAN: Ipinabatid ni Presidenti...,INVALID,INVALID,MOSTLY_TRUE
4,PIACaraga,2021-09-21 23:52:23+00:00,"RT @pcoogov: PANOORIN: Kahapon, Setyembre 20, ...",INVALID,TRUE,TRUE
...,...,...,...,...,...,...
9132,moonchild_kni21,2021-09-16 01:39:07+00:00,fvcking coronavirus,INVALID,INVALID,INVALID
9133,furaaanzu,2021-09-16 01:39:03+00:00,"""oh sige, ikaw na! sayo na ang korona kasama a...",INVALID,MOSTLY_FALSE,INVALID
9134,GMARTVNews,2021-09-16 01:25:00+00:00,"ALBAY — COVID-19 surge plan, inilatag na ng DO...",TRUE,TRUE,TRUE
9135,GMARTVNews,2021-09-16 01:15:00+00:00,"CALASIAO, PANGASINAN — Ilang pasyente sa Calas...",TRUE,TRUE,TRUE


In [ ]:
df = df.drop_duplicates(keep='first', subset=['tweet'])
df = df.reset_index()
df = df.rename(columns={'index':'orig_index'})
df

,orig_index,username,created_at,tweet,verdict_neilson,verdict_michelle,verdict_liecel
0,0,PIACaraga,2021-09-21 23:53:36+00:00,"RT @pcoogov: TINGNAN: Kahapon, Setyembre 20, i...",INVALID,TRUE,TRUE
1,1,PIACaraga,2021-09-21 23:53:27+00:00,RT @pcoogov: TINGNAN: Nakatanggap ng additiona...,INVALID,TRUE,TRUE
2,2,PIACaraga,2021-09-21 23:53:17+00:00,"RT @pcoogov: TINGNAN: Nitong Lunes, Setyembre ...",INVALID,INVALID,TRUE
3,3,PIACaraga,2021-09-21 23:52:36+00:00,RT @pcoogov: TINGNAN: Ipinabatid ni Presidenti...,INVALID,INVALID,MOSTLY_TRUE
4,4,PIACaraga,2021-09-21 23:52:23+00:00,"RT @pcoogov: PANOORIN: Kahapon, Setyembre 20, ...",INVALID,TRUE,TRUE
...,...,...,...,...,...,...,...
4158,9132,moonchild_kni21,2021-09-16 01:39:07+00:00,fvcking coronavirus,INVALID,INVALID,INVALID
4159,9133,furaaanzu,2021-09-16 01:39:03+00:00,"""oh sige, ikaw na! sayo na ang korona kasama a...",INVALID,MOSTLY_FALSE,INVALID
4160,9134,GMARTVNews,2021-09-16 01:25:00+00:00,"ALBAY — COVID-19 surge plan, inilatag na ng DO...",TRUE,TRUE,TRUE
4161,9135,GMARTVNews,2021-09-16 01:15:00+00:00,"CALASIAO, PANGASINAN — Ilang pasyente sa Calas...",TRUE,TRUE,TRUE


In [ ]:
df['verdict'] = 'INVALIDT'
count = 0

for i in df.index:
  vc = {'TRUE':0,'MOSTLY_TRUE':0, 'HALF_TRUE':0,'MOSTLY_FALSE':0, 'FALSE':0,'INVALID':0, 'OUT_OF_SCOPE':0}
  vc[df.loc[i].verdict_neilson] = vc[df.loc[i].verdict_neilson] + 1
  vc[df.loc[i].verdict_michelle] = vc[df.loc[i].verdict_michelle] + 1
  vc[df.loc[i].verdict_liecel] = vc[df.loc[i].verdict_liecel] + 1
  vc = dict(sorted(vc.items(), key=lambda item: item[1]))
  last_key = list(vc.keys())[-1]
  if vc[last_key] > 1:
    df.at[i,'verdict'] = last_key
    count += 1


df

,orig_index,username,created_at,tweet,verdict_neilson,verdict_michelle,verdict_liecel,verdict
0,0,PIACaraga,2021-09-21 23:53:36+00:00,"RT @pcoogov: TINGNAN: Kahapon, Setyembre 20, i...",INVALID,TRUE,TRUE,TRUE
1,1,PIACaraga,2021-09-21 23:53:27+00:00,RT @pcoogov: TINGNAN: Nakatanggap ng additiona...,INVALID,TRUE,TRUE,TRUE
2,2,PIACaraga,2021-09-21 23:53:17+00:00,"RT @pcoogov: TINGNAN: Nitong Lunes, Setyembre ...",INVALID,INVALID,TRUE,INVALID
3,3,PIACaraga,2021-09-21 23:52:36+00:00,RT @pcoogov: TINGNAN: Ipinabatid ni Presidenti...,INVALID,INVALID,MOSTLY_TRUE,INVALID
4,4,PIACaraga,2021-09-21 23:52:23+00:00,"RT @pcoogov: PANOORIN: Kahapon, Setyembre 20, ...",INVALID,TRUE,TRUE,TRUE
...,...,...,...,...,...,...,...,...
4158,9132,moonchild_kni21,2021-09-16 01:39:07+00:00,fvcking coronavirus,INVALID,INVALID,INVALID,INVALID
4159,9133,furaaanzu,2021-09-16 01:39:03+00:00,"""oh sige, ikaw na! sayo na ang korona kasama a...",INVALID,MOSTLY_FALSE,INVALID,INVALID
4160,9134,GMARTVNews,2021-09-16 01:25:00+00:00,"ALBAY — COVID-19 surge plan, inilatag na ng DO...",TRUE,TRUE,TRUE,TRUE
4161,9135,GMARTVNews,2021-09-16 01:15:00+00:00,"CALASIAO, PANGASINAN — Ilang pasyente sa Calas...",TRUE,TRUE,TRUE,TRUE


In [ ]:
print(count)
df = df.drop(columns=['verdict_neilson','verdict_michelle','verdict_liecel'])

3796


In [ ]:
df.verdict.value_counts()

INVALID         1577
TRUE            1267
OUT_OF_SCOPE     486
INVALIDT         367
MOSTLY_TRUE      306
MOSTLY_FALSE      71
FALSE             59
HALF_TRUE         30
Name: verdict, dtype: int64

In [ ]:
for i in df.index:
  if df.loc[i].verdict == 'OUT_OF_SCOPE':
    df.at[i,'verdict'] = 'INVALID' 

In [ ]:
df.to_csv(filename_export,index=False)

In [ ]:
df.verdict.value_counts()

INVALID         2063
TRUE            1267
INVALIDT         367
MOSTLY_TRUE      306
MOSTLY_FALSE      71
FALSE             59
HALF_TRUE         30
Name: verdict, dtype: int64